### Задание 1.  

Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)  
2. Сделать нейронную сеть  
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)  
4. Проверить качество

In [17]:
import torch
import re
import pandas as pd

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [4]:
def caesar(word, key):
#    wordl = word.lower()
    encrypted = []
    for letter in word:
        if letter in alphabet:
            new_key = (alphabet.find(letter) + key)  % len(alphabet)
            encrypted += alphabet[new_key]
        else:
            encrypted += letter
    return encrypted

In [5]:
print(caesar('приветя', 5))

['ф', 'х', 'н', 'ж', 'й', 'ч', 'д']


In [6]:
with open('Архив/onegin.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^а-я]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 44586


In [7]:
onegin = caesar(text, 1)
print(text[:15])
print(onegin[:15])

евгений онегин 
['ё', 'г', 'д', 'ё', 'о', 'й', 'к', ' ', 'п', 'о', 'ё', 'д', 'й', 'о', ' ']


In [8]:
INDEX_TO_CHAR = ['none'] + [w for w in alphabet]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [9]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [10]:
Y = torch.zeros((len(onegin), MAX_LEN), dtype=int)
for i in range(len(onegin)):
    for j, w in enumerate(onegin[i]):
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [11]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear= torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, word, state=None):
        embed = self.embed(word)
        o, s = self.rnn(embed, state)
        out = self.linear(o)
        return out, s

In [12]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [13]:
for ep in range(10):
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Train loss: {:.3f}".format(ep, train_loss / train_passed))

Epoch 0. Train loss: 0.055
Epoch 1. Train loss: 0.011
Epoch 2. Train loss: 0.005
Epoch 3. Train loss: 0.003
Epoch 4. Train loss: 0.002
Epoch 5. Train loss: 0.001
Epoch 6. Train loss: 0.001
Epoch 7. Train loss: 0.001
Epoch 8. Train loss: 0.001
Epoch 9. Train loss: 0.001


### Задание 2.  
Выполнить практическую работу из лекционного ноутбука.

1. Построить RNN-ячейку на основе полносвязных слоев  
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [18]:
df = pd.read_csv('Архив/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [19]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [20]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [22]:
text[1]

['l', 'e', 'e', '-', 'm', 'u', 'r', ' ', 'l', 'e', 'e', '-', 'm', 'u', 'r']

In [23]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [24]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [26]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128)
        self.drop = torch.nn.Dropout(0,5)
        self.out= torch.nn.Linear(128, 28)
        
    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        self.relu = torch.nn.ReLU()
        x, s = self.rnn(x)
        return self.out(x)

In [27]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [31]:
import time

In [33]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    
    x = torch.zeros((1, len(sentence)), dtype=int)
    
    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        else:
            x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w]
        
        if ww == 'none':
            break
            
        sentence.append(ww)
    
    print(' '.join(sentence))

In [43]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 2.548, Train loss: 1.617
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 1. Time: 2.873, Train loss: 1.616
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 2. Time: 2.451, Train loss: 1.616
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 3. Time: 2.626, Train loss: 1.616
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 4. Time: 2.462, Train loss: 1.615
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 5. Time: 2.360, Train loss: 1.615
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e
Epoch 6. Time: 2.342, Train loss: 1.615
h e l l o n g   t h e   t h e   t h e   t h e   t h e   t h 

Запускал много раз, ошибка падает, но текс свалился в the и не меняется.